## pyTopoComplexity (v0.5.4)
### **Mexican Hat Wavelet Analysis** *[Base Version]*  

Two-dimensional continuous wavelet transform (2D-CWT) with a Mexican Hat wevalet has been applied to measure the topographic complexity (i.e., surface roughness) of a land surface. Such method quanitfy the wavelet-based curvature of the surface, which has been proposed to be a effective geomorphic metric for relative age dating of deep-seated landslide deposits, allowing a quick assessment of landslide freqency and spatiotemporal pattern over a large area.

The original MATLAB code was developed by Dr. Adam M. Booth (Portland State Univeristy) and used in Booth et al. (2009) and Booth et al. (2017). This MATLAB code was later revised and adapted by Dr. Sean R. LaHusen (Univeristy of Washington) and Dr. Erich N. Herzig (Univeristy of Washington) in their research (e.g., LaHusen et al., 2020; Herzig et al., 2023).

Since November 2023, Dr. Larry Syu-Heng Lai (Univeristy of Washington) translated this code into a open-source Python version with continous optimizations. The current codes have the capability to automoatically detect the grid spacing ($\Delta$) and the unit of XYZ direction of the input Digital Elevation Model (DEM) raster and compute the 2D-CWT results with the adequate wavelet scale factor ($s$) at an designated Mexican Hat wavelet ($\lambda$).

The example GeoTIFF rasters include the LiDAR Digital Elevation Model (DEM) files that cover the area and nearby region of a deep-seated landslide occurred in 2014 at Oso area of the North Fork Stillaguamish River (NFSR) valley, Washington State, USA (Washington Geological Survey, 2023). The example DEMs have various grid size, coordinate reference system (CRS), and unit of grid value (elevation, Z). 

To use this code, please cite the Zenodo repository that hosts the latest release of this code: 
* Lai, L. S.-H. (2024). pyTopoComplexity. Zenodo. https://doi.org/10.5281/zenodo.11239338
* Github repository: https://github.com/LarrySHLai/pyTopoComlexity
<hr>

### **Theory** (Booth et al., 2009; Torrence and Compo, 1998)

The 2D-CWT provides information regarding how amplitude is distributed over spatial frequency at each position in the data by transforming spatial data into position-frequency space. The 2D-CWT is calculated by a convolution of the elevation $z$ and a wavelet family $\psi$, with a wavelet scale parameter $s$ at every location ($x$, $y$):
$$
C (s, x, y) = \Delta^2 \int_{-\infty}^{\infty} \int_{-\infty}^{\infty} z(x, y) \psi \left( x, y \right) dx \, dy
$$

, where the wavelet coefficient $C(s,x,y)$ provides a measure of how well the wavelet $\psi$ matches the data $z$ at each node/grid. When $s$ is large, $\psi$ is spread out and takes into account long wavelength features of $z$; when $s$ is small, $\psi$ is more localized in space and sensitive to fine-scale features of $z$. 

Here we use 2D Mexican hat wavelet function to describe $\psi$:

$$
\psi = − \frac{1}{\pi(s\Delta)^4}(1-\frac{𝑥^2+𝑦^2}{2s^2})e^{(-\frac{𝑥^2+𝑦^2}{2s^2})}
\:\:\:\:\:\:\:\:\:
\lambda=\frac{2\pi s}{\sqrt{5/2}}\Delta
$$

The Mexican hat is proportional to the second derivative of a Gaussian envelope, and it has a wavelength ($\lambda$) which is dependent on the grid spacing ($\Delta$) of the input raster. The $\psi$ here has been scaled to the wavelet scale parameter $s$ and the grid spacing $\Delta$ so that the wavelet coefficient $C$ is equal to curvature. 
<hr>

### **References**
##### Journal Articles: 
* Booth, A.M., LaHusen, S.R., Duvall, A.R., Montgomery, D.R., 2017. Holocene history of deep-seated landsliding in the North Fork Stillaguamish River valley from surface roughness analysis, radiocarbon dating, and numerical landscape evolution modeling. Journal of Geophysical Research: Earth Surface 122, 456-472. https://doi.org/10.1002/2016JF003934 
* Booth, A.M., Roering, J.J., Perron, J.T., 2009. Automated landslide mapping using spectral analysis and high-resolution topographic data: Puget Sound lowlands, Washington, and Portland Hills, Oregon. Geomorphology 109, 132-147. https://doi.org/10.1016/j.geomorph.2009.02.027   
* Herzig, E.N., Duvall, A.R., Booth, A.R., Stone, I., Wirth, E., LaHusen, S.R., Wartman, J., Grant, A., 2023. Evidence of Seattle Fault Earthquakes from Patterns in Deep‐Seated Landslides. Bulletin of the Seismological Society of America. https://doi.org/10.1785/0120230079 
* LaHusen, S.R., Duvall, A.R., Booth, A.M., Grant, A., Mishkin, B.A., Montgomery, D.R., Struble, W., Roering, J.J., Wartman, J., 2020. Rainfall triggers more deep-seated landslides than Cascadia earthquakes in the Oregon Coast Range, USA. Science Advances 6, eaba6790. https://doi.org/10.1126/sciadv.aba6790  
* Torrence, C., Compo, G.P., 1998. A practical guide to wavelet analysis. Bulletin of the American Meteorological Society 79 (1), 61–78.

##### Digital Elevation Model (DEM) Examples:
* Washington Geological Survey, 2023. 'Stillaguamish 2014' and 'Snohoco Hazel 2006' projects [lidar data]: originally contracted by Washington State Department of Transportation (WSDOT). [accessed April 4, 2024, at http://lidarportal.dnr.wa.gov]
<hr>

#### 0. Import packages

In [ ]:
import os
import numpy as np
from scipy.signal import convolve2d
from scipy.signal import fftconvolve
import rasterio
import matplotlib.pyplot as plt
from matplotlib.colors import LightSource
from ipywidgets import interactive, FloatSlider

#### 1. Define the 2D-CWT function

Here the function `conv_mexh` operates the 2D-CWT convolution via **numpy** packages. The conventional way is using <code>convolve2d</code> - A much slower method. A optimized way is using <code>fftconvolve</code> Fast Fourier Transform (FFT) - Much faster method

***Notes:***
If the input DEM raster include grids that contain uncommonly/errorenously created no-data values, the <code>fftconvolve</code> package used in the <code>conv2_mexh</code> function will return empty result in that chunk with an error message *"RuntimeWarning: invalid value encountered in multiply ret = ifft(sp1 * sp2, fshape, axes=axes)"*. The function will still proceed to process the rest of the chunks, resulting a raster with empty squares. In this case, users may switch to the conventional <code>convolve2d</code> convolution package in the <code>conv2_mexh</code>, which will significantlly increase the processing time.

In [ ]:
def conv2_mexh(Z, s, Delta):
    # The kernel must be large enough for the wavelet to decay to ~0 at the edges.
    X, Y = np.meshgrid(np.arange(-8 * s, 8 * s + 1), np.arange(-8 * s, 8 * s + 1))

    # scaled psi equation. Units of [1/(m^4)]
    psi = (-1/(np.pi*(s * Delta)**4)) * (1 - (X**2 + Y**2)/(2 * s**2)) * np.exp(-(X**2 + Y**2)/(2* s**2))  

    # Calculating the wavelet coefficient C, multiplied by Delta^2. Units of [(m^2) x (m) x (1/(m^4)) = (1/m)]
    # 'same' mode is used to approximate the double integral. 
    #C = (Delta**2) * convolve2d(Z, psi, mode='same') #Slow conventional method
    C = (Delta**2) * fftconvolve(Z, psi, mode='same') #Fast Fourier Transform (FFT) method

    return C

#### 2. Set up the input and output file names and directories
The default assumes the input and output GeoTIFF rasters will be placed in the same directory

In [ ]:
base_dir = os.getcwd()  #Change the directory of base folder as needed
base_dir = os.path.join(base_dir, 'Example DEM')

input_file = 'Ososlid2014_m_6ftgrid.tif'
output_file = 'Ososlid2014_m_6ftgrid_pymexhat.tif'

input_dir = os.path.join(base_dir, input_file)
output_dir = os.path.join(base_dir, output_file)

#### 3. Extract information from the input DEM raster
The following section is used to extract coordinate reference system (crs) information, which is critical to contrain the grid spacing $\Delta$ and wavelet scale $s$ 

In [ ]:
with rasterio.open(input_dir) as src:
    transform = src.transform
    crs = src.crs

    # Print the CRS information
    print(f"CRS as WKT: {crs.wkt}")
    print(f"CRS as EPSG code: {crs.to_epsg()}")
    print(f"X grid size: {transform[0]} [{crs.linear_units}]")
    print(f"Y grid size: {-transform[4]} [{crs.linear_units}]")

#### 4. Define parameters
Define the desired Mexican Hat Fourier wavelet $\lambda$ (in meters)

In [ ]:
Lambda = 15  #meters

Derive the correct grid spacing $\Delta$ and wavelet scale $s$

In [ ]:
ft2mUS = 1200/3937 #US survey foot to meter conversion factor 
ft2mInt = 0.3048   #International foot to meter conversion factor 

def Delta_s_Calculate(input_dir, Lambda):
    with rasterio.open(input_dir) as src:
        transform = src.transform
        crs = src.crs   

    # Delta is the grid spacing (pixel size) of the input DEM raster. Unit in [m].
    if any(unit in crs.linear_units.lower() for unit in ["metre".lower(), "meter".lower()]):
        print("Input grid size is in meters. No unit conversion is made")
        Delta = np.mean([transform[0], -transform[4]])
    elif any(unit in crs.linear_units.lower() for unit in ["foot".lower(), "feet".lower(), "ft".lower()]):  
        if any(unit in crs.linear_units.lower() for unit in ["US".lower(), "United States".lower()]):
            print("Input grid size is in US survey feet. A unit conversion to meters is made")
            Delta = np.mean([transform[0] * ft2mUS, -transform[4] * ft2mUS])
        else: 
            print("Input grid size is in international feet. A unit conversion to meters is made")
            Delta = np.mean([transform[0] * ft2mInt, -transform[4] * ft2mInt])
    else:
        message = (
        "WARNING: The code excution is stopped. "
        "The units of XY directions must be in feet or meters. "
        "Please reproject the raster into a suitable coordinates reference system."
        )
        raise RuntimeError(message)

    # 's' is the scale of the wavelet [Unitless] 
    s = (Lambda/Delta)*((5/2)**(1/2)/(2*np.pi))      # Torrence and Compo [1998]
    # When Delta close to 1 meter, 's' aprox. mex-hat wavelength/4

    print('Grid spacing "Delta" =', Delta, '[m]')
    print('For a', Lambda, 'm Mexican Hat wavelet, ...')
    print('the wavelet scale "s" =', s, '[]')

    return Delta, s

Delta, s = Delta_s_Calculate(input_dir, Lambda)

#### 5. Function of processing the input GeoTIFF

In [ ]:
ft2mUS = 1200/3937 #US survey foot to meter conversion factor 
ft2mInt = 0.3048 #International foot to meter conversion factor 

def process_mexhat(input_dir, s, Delta):
    with rasterio.open(input_dir) as src:
        Z = src.read(1)
        Zunit = src.crs.linear_units   #assuming the Z unit is the same as the units of XY directions
        #Zunit = "metre"               #Manually define the elevation unit. Acceptable inputs: "metre", "meter", "meters", "foot", "feet", "ft", "US survey foot"

    # Check the unit of Z and make unit conversion when needed
    if any(unit in Zunit.lower() for unit in ["metre".lower(), "meter".lower()]):
        print("Input elevation is in meters. No unit conversion is made")
    elif any(unit in Zunit.lower() for unit in ["foot".lower(), "feet".lower(), "ft".lower()]):  
        if any(unit in Zunit.lower() for unit in ["US".lower(), "United States".lower()]):
            print("Input elevation is in US survey feet. A unit conversion to meters is made")
            Z = Z * ft2mUS
        else:
            print("Input elevation is in international feet. A unit conversion to meters is made")
            Z = Z * ft2mInt
    else:
        message = (
        "WARNING: The code excution is stopped. "
        "The unit of elevation 'z' must be in feet or meters. "
        "Please redefine the 'Zunit' parameter."
        )
        raise RuntimeError(message)
        
    # Compute Mexican Hat 2D Continuous Wavelet Transform
    C2 = conv2_mexh(Z, s, Delta)
    result = np.abs(C2)

    # Mask edge with NaN (no data) values to remove artifacts
    cropedge = np.ceil(s * 4)
    fringeval = int(cropedge)
    result[:fringeval, :] = np.nan
    result[:, :fringeval] = np.nan
    result[-fringeval:, :] = np.nan
    result[:, -fringeval:] = np.nan

    print(f"The Mexican Hat convolution is done")

    return Z, result

#### 6. Executing 2D-CWT and export the outcome raster

In [ ]:
Z, result = process_mexhat(input_dir, s, Delta)

# Write result to GeoTIFF
with rasterio.open(output_dir, 'w', driver='GTiff', height=Z.shape[0], 
                width=Z.shape[1], count=1, compress='deflate',
                    bigtiff='IF_SAFER', dtype=Z.dtype, crs=crs, transform=transform) as dst:
    dst.write(result.astype(Z.dtype), 1)

print(f"Processed {os.path.basename(input_dir)} and saved to {os.path.basename(output_dir)}")

#### 7. Display the result
Plot hillshade and the 2D-CWT topographic roughness result

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 6))

# Plot the hillshade
ls = LightSource(azdeg=315, altdeg=45)
hs = axes[0].imshow(ls.hillshade(Z, vert_exag=2), cmap='gray') #2x vertical exaggeration for hillshade 
axes[0].set_title(input_file)
axes[0].set_xlabel(f'X-axis grids \n(grid size ≈ {round(transform[0],4)} [{crs.linear_units}])')
axes[0].set_ylabel(f'Y-axis grids \n(grid size ≈ {-round(transform[4],4)} [{crs.linear_units}])')
cbar1 = fig.colorbar(hs, ax=axes[0], orientation='horizontal', fraction=0.045, pad=0.13)
cbar1.ax.set_visible(False)

# Plot the 2D-CWT roughness
im = axes[1].imshow(result, cmap='jet')
im.set_clim(0, round(np.nanpercentile(result, 99), 2))  # Set the upperlimit to the 99th percentile to avoid exterem values
axes[1].set_title(output_file)
axes[1].set_xlabel(f'X-axis grids \n(grid size ≈ {round(transform[0],4)} [{crs.linear_units}])')
axes[1].set_ylabel(f'Y-axis grids \n(grid size ≈ {-round(transform[4],4)} [{crs.linear_units}])')
cbar2 = fig.colorbar(im, ax=axes[1], orientation='horizontal', fraction=0.045, pad=0.13)
cbar2.set_label(f'Mexican Hat {Lambda} m 2D-CWT surface roughness [m$^{{-1}}$]')
plt.tight_layout()
plt.show()

#### EXTRA. Interactive display
Adjust $\lambda$ value to see the change or 2D-CWT measurement result. The opacity of 2D-CWT image overlay is adjustable as well.

In [ ]:
def blend_images(alpha, Lambda):

    Lambda = round(Lambda,1)

    Delta, s = Delta_s_Calculate(input_dir, Lambda)
    Z, result = process_mexhat(input_dir, s, Delta)

    # Visualization
    fig, ax = plt.subplots(figsize=(10, 6))
    ls = LightSource(azdeg=315, altdeg=45)
    hillshade_image = ls.hillshade(Z, vert_exag=2)
    hs = ax.imshow(hillshade_image, cmap='gray', alpha=1-alpha)  # Adjust alpha for blending
    roughness_image = result
    max_val = round(np.nanpercentile(roughness_image, 99), 2)
    im = ax.imshow(roughness_image, cmap='jet', alpha=alpha, vmin=0, vmax=max_val)

    ax.set_title(f'Mexican Hat Wavelet (λ) = {Lambda} m')

    with rasterio.open(input_dir) as src:
        ax.set_xlabel(f'X-axis grids \n(grid size ≈ {round(src.transform[0],4)} [{src.crs.linear_units}])')
        ax.set_ylabel(f'Y-axis grids \n(grid size ≈ {-round(src.transform[4],4)} [{src.crs.linear_units}])')
    
    cbar = fig.colorbar(im, ax=ax)
    cbar.set_label(f'2D-CWT surface roughness [m$^{{-1}}$]')

    plt.tight_layout()
    plt.show()

# Define default values and slider configurations
default_alpha = 0.35
default_lambda = 15
alpha_slider = FloatSlider(value=default_alpha, min=0.0, max=1.0, step=0.1, description='Opacity', style={'description_width': 'initial'})
lambda_slider = FloatSlider(value=default_lambda, min=10, max=100, step=5, description=f'λ (m)', style={'description_width': 'initial'})

# Create an interactive widget with configured sliders
interactive(blend_images, alpha=alpha_slider, Lambda=lambda_slider)